In [ ]:
%load_ext kamu

In [ ]:
%import_dataset us.cityofnewyork.data.zipcode-boundaries --alias zipcodes

In [ ]:
zipcodes.printSchema()

In [ ]:
%%sql
select po_name, sum(population) as population from zipcodes group by po_name order by po_name

In [ ]:
%%sql
select po_name, sum(population) as population from zipcodes group by po_name order by po_name

In [ ]:
%%sql -o count_per_zipcode
SELECT 
    zipcode,
    last(po_name) as name,
    sum(population) / sum(area) as density,
    last(ST_AsText(ST_Transform(geometry, "epsg:2263", "epsg:4326"))) as geometry
FROM zipcodes
GROUP BY zipcode

In [ ]:
%%local
import shapely.wkt
import geojson

def df_to_geojson(df, geom='geometry', props=None):
    if props is None:
        props = [
            c for c in df.columns
            if c != geom
        ]

    return geojson.FeatureCollection([
        geojson.Feature(
            geometry=shapely.wkt.loads(r[geom]),
            properties={p: r[p] for p in props}
        )
        for _, r in df.iterrows()
    ])

In [ ]:
%%local
gj = df_to_geojson(count_per_zipcode)

In [ ]:
%%local
import os
from mapboxgl.viz import *
from mapboxgl.utils import *

# Must be a public token, starting with `pk`
token = os.getenv('MAPBOX_ACCESS_TOKEN')

In [ ]:
%%local

num_max = count_per_zipcode.density.max()
stops = [i * num_max / 4 for i in range(4)]

viz = ChoroplethViz(
    gj, style='mapbox://styles/mapbox/dark-v10', center=(-74.068266, 40.7019673), zoom=9,
    access_token=token,
    color_property='density',
    color_stops=create_color_stops(stops, colors='YlOrRd'),
    color_function_type='interpolate',
    line_stroke='solid',
    line_width=0.3,
    line_color='rgb(128,0,38)',
    opacity=0.8,)
viz.show()